In [2]:
import numpy as np
import os
import SimpleITK as sitk

In [3]:
def get_lists_of_of_paths(directory):
    file_list = os.listdir(directory)
    file_paths = []
    for file in file_list:
        file_path = os.path.join(directory, file)
        file_paths.append(file_path)

    return file_paths  


def fn_scan_to_array(base_path):
    mid_file=get_lists_of_of_paths(base_path)[0]
    slice_list=get_lists_of_of_paths(mid_file)
    for i in range(160):
        if i==0:    
            slice = sitk.ReadImage(get_lists_of_of_paths(mid_file)[0])
            scan_array3D = (sitk.GetArrayFromImage(slice)).reshape(384,384)
        else:  
            slice = sitk.ReadImage(get_lists_of_of_paths(mid_file)[i])
            slice_array= sitk.GetArrayFromImage(slice).reshape(384,384)
            scan_array3D=np.dstack((scan_array3D, slice_array))
    return scan_array3D  # (384, 384, 160)



def fn_segm_mask_to_array(subject_name):

    mhd_path = "data/" + subject_name +"/"+subject_name+".segmentation_masks.mhd"
    segm_mask = sitk.GetArrayFromImage(sitk.ReadImage(mhd_path, sitk.sitkFloat32))
    return np.flip(segm_mask, axis=0) # (384, 384, 160)


In [7]:
from utils import getAverageMask, structure_tensor_3D, getGradients
from trimesh.creation import icosphere
from sklearn.utils.extmath import cartesian
from sklearn.metrics.pairwise import linear_kernel
import math
import itertools
from sklearn.preprocessing import normalize
from scipy.stats import kurtosis
from scipy.special import sph_harm
from scipy.ndimage.interpolation import map_coordinates

C:\Users\ChrisIoannidis\AppData\Local\Temp\ipykernel_18920\3037991853.py:10: DeprecationWarning: Please use `map_coordinates` from the `scipy.ndimage` namespace, the `scipy.ndimage.interpolation` namespace is deprecated.
  from scipy.ndimage.interpolation import map_coordinates


In [8]:
def lbp_ri_sh(point, img, patch_size, sph_degree, ico_radius, ico_level, n_bins = 30, concatenate = True):

    psize = patch_size // 2
    x, y, z = point
    patch = img[x - psize - int(math.ceil(ico_radius)) : x + psize + int(math.ceil(ico_radius)) + 1,
                y - psize - int(math.ceil(ico_radius)) : y + psize + int(math.ceil(ico_radius)) + 1,
                z - psize - int(math.ceil(ico_radius)) : z + psize + int(math.ceil(ico_radius)) + 1]
    
    patch_coords = cartesian((range(patch_size), range(patch_size), range(patch_size))) + 1    

    #construct icosahedron for uniform sampling on sphere surface
    ico = icosphere(subdivisions = ico_level, radius = ico_radius)
    ico_coords = np.array(ico.vertices)
    theta = np.arccos(ico_coords[:, 2] / ico_radius)
    phi = np.arctan2(ico_coords[:, 1], ico_coords[:, 0])

    #get Spherical Harmonics expansion coefficients (up to degree sph_degree)
    m = list(itertools.chain.from_iterable([[i for i in range(-n,n+1)] for n in range(sph_degree)]))
    m = np.array(m)

    l = list(itertools.chain.from_iterable([[k for _ in range(2 * k + 1)] for k in range(sph_degree)]))
    l = np.array(l)

    Y = sph_harm(m[None, :], l[None, :], theta[:, None], phi[:, None])

    #sample sphere neighbors for each voxel in patch and interpolate intensity
    mapped_coords = patch_coords[None, :, :] + ico_coords[:, None, :]
    mapped_int = map_coordinates(patch, mapped_coords.T, order = 3)
    center_int = patch[ico_radius : -ico_radius, ico_radius : -ico_radius, ico_radius : -ico_radius]

    #Compute kurtosis (for better rotation invariance)
    kurt = kurtosis(mapped_int)

    #Apply sign function and pass obtain spherical expansion coefficients for each sample
    f = np.greater_equal(center_int.ravel()[:, None], mapped_int).astype('int')
    c = f.dot(Y)

    #obtain frequency components of threshold function by integrating and normalizing over orders m
    f = np.multiply(c[:, None, l == 0], Y[None, :, l == 0])
    for n in range(1, sph_degree):
        f = np.concatenate((f, np.sum(np.multiply(c[:, None, l == n], Y[None, :, l == n]),
                            axis=2, keepdims=True)), axis=2)
    f = np.sqrt(np.sum(f**2, axis=1))

    #keep real parts of decomposition and kurtosis
    f = np.real(f)
    kurt = np.real(kurt)

    #extract histograms
    H = np.histogram(kurt, bins = n_bins)[0]
    for i in range(sph_degree):
        H = np.column_stack((H, np.histogram(f[:, i], bins = n_bins)[0]))
    H = normalize(H, axis = 0)

    #Return Descriptor (concatenated or aggregated histograms)
    if concatenate is True:
        D = H.T.ravel()
    else: 
        D = H.sum(axis = 1)
    D = D / np.linalg.norm(D)

    return D





def hog_3d_proj(point, image, psize = 5, rsize = 15, orientation = 'vertices', level = 1, mode = 'concatenated', 
                rot_inv = False, norm = 'l2'):
    '''
    Computes a 3D variant of the HOG Descriptor for an image region centered arounda voxel
    The Region of size (rsize x rsize x rsize) is compartmentalized into a set of disjoint patches,
    each of size (psize x psize x psize). A histogram of oriented gradients is computed for each patch, 
    with the orientation bins corresponding to vertices of centers of faces of a regular (refined) icosahedron.
    The final descriptor is a weighted aggregate of those histograms. Currently, implementation supports regions arranged in
    3x3x3 patches.

    Reference: Alexander Klaser, Marcin Marszalek, Cordelia Schmid. 
               A Spatio-Temporal Descriptor Based on 3D-Gradients. 
               BMVC 2008 - 19th British Machine Vision Conference, Sep 2008, Leeds, United Kingdom.pp.275:1-10. 
               DOI:10.5244/C.22.99

    ...

    Parameters
    ----------
    point : array - like
        the voxels to be characterized
    image : ndarray
        the image containing the voxels
    psize : int
        size of patches in region
    rsize : int
        size of region around central voxel
    orientation : string
        whether to associate histogram bins with vertices of centroids of faces of the icosahedron
    ico_coords : string
        coordinate system to define icosahedron on
    level : int
        number of refienement steps for icosahedron
    mode : string
        chooses whether to concatenate or aggregate patch histograms to form final descriptor
    
    Returns
    -------
    D : ndarray
        voxel descriptor

    '''

    #sanity check
    assert type(rsize // psize) == int, print("Wrong combination of regional and patch sizes")

    #set params
    rs = rsize // 2
    ps = psize // 2
    ncells = rsize // psize
    
    # get icosahedron
    ico = icosphere(subdivisions = level)
    if orientation  == 'faces':
        axes = np.array(ico.face_normals)
    else:
        axes = np.array(ico.vertices)

    # get average masks
    region_mask = getAverageMask(rsize // psize, 'manhattan')
    patch_mask = getAverageMask(psize, 'manhattan')

    #calculate partial derivatives
    x, y, z = point 
    xp = range(- rs + ps, rs - ps + 1, psize)
    yp = range(- rs + ps, rs - ps + 1, psize)
    zp = range(- rs + ps, rs - ps + 1, psize)
    patch_centers = cartesian((xp, yp, zp))
    patch_locations = patch_centers + psize

    # extracting +1 voxel in each direction for computational consistency
    region = image[x - rs - 1 : x + rs + 2,
                   y - rs - 1 : y + rs + 2,
                   z - rs - 1 : z + rs + 2]
        
    i_dx, i_dy, i_dz = getGradients(region)

    #get gradients at the patch level
    dx = np.array([i_dx[ploc[0] : ploc[0] + psize, ploc[1] : ploc[1] + psize,
                        ploc[2] : ploc[2] + psize] for ploc in patch_locations])

    dy = np.array([i_dy[ploc[0] : ploc[0] + psize, ploc[1] : ploc[1] + psize,
                        ploc[2] : ploc[2] + psize] for ploc in patch_locations])
    
    dz = np.array([i_dz[ploc[0] : ploc[0] + psize, ploc[1] : ploc[1] + psize,
                        ploc[2] : ploc[2] + psize] for ploc in patch_locations])

    dx = dx.reshape((ncells**3, psize**3))
    dy = dy.reshape((ncells**3, psize**3))
    dz = dz.reshape((ncells**3, psize**3))

    #collect all gradients in one array and calculate magnitudes
    raw_gradients = np.dstack((dx, dy, dz))
    if rot_inv is True:
        #rotate region according to dominant direction to achieve rotational invariance
        R = structure_tensor_3D(raw_gradients, getAverageMask(rsize, 'gaussian'))
        gradients = R.T.dot(raw_gradients.reshape((-1, 3)).T) 
        gradients = gradients.reshape(3, raw_gradients.shape[1], raw_gradients.shape[0]).T
    else:
        gradients = raw_gradients
    gradient_magnitudes = np.linalg.norm(gradients, axis = 2)

    #project gradients to icosahedron orientation axes
    projected_gradients = gradients.dot(axes.T)
    projected_gradients /= (gradient_magnitudes[:, :, None]+0.001)

    # compute theshold to clip projected gradients and recalculate magnitude
    inner_prods = linear_kernel(axes)[0, :]
    thres = np.sort(inner_prods)[-2]

    projected_gradients -= thres
    projected_gradients[projected_gradients < 0] = 0
    projected_gradient_magnitudes = np.linalg.norm(projected_gradients, axis = 2)

    #distribute original magnitude in orientation bins
    gradient_histograms = projected_gradients * (gradient_magnitudes[:, :, None] / (projected_gradient_magnitudes[:, :, None]+0.001))
    D = gradient_histograms.sum(axis = 1)

    if mode == 'flatten':
        Descriptor = (region_mask.ravel()[:, None] * D).ravel()
    else:
        Descriptor = region_mask.ravel().dot(D)

    if norm == 'l2':
        Descriptor = Descriptor / np.linalg.norm(Descriptor)
    if norm == 'l2-hys':
        Descriptor = Descriptor / np.linalg.norm(Descriptor)
        Descriptor = np.clip(Descriptor, a_min = 0, a_max = 0.25)
        Descriptor = Descriptor / np.linalg.norm(Descriptor)
    
    return Descriptor
   

def fn_hog_lbp_descriptor(coordinate, array):  
    descriptor_lbp = lbp_ri_sh(coordinate, array, 5, 4, 2, 2, concatenate = True)
    descriptor_hog = hog_3d_proj(coordinate, array) 
    descriptor_lbp_hog = np.concatenate([descriptor_hog,descriptor_lbp])
    return descriptor_lbp_hog

In [9]:
codewords = np.load('codewords.npy')
mri_scan = fn_scan_to_array('data/9001104')

In [50]:
subregion_descriptors = []
for coordinate in np.load('coordinates/sub_3_coord.npy'):
    subregion_descriptors.append(np.array(fn_hog_lbp_descriptor(coordinate, mri_scan)))

subregion_descriptors = np.array(subregion_descriptors)

In [47]:


def soft_assign_histogram(codewords, subregion_descriptors, a = 5):
    histogram = np.zeros(codewords.shape[0])
    for j, descriptor in enumerate(subregion_descriptors):
        descriptor_assignment=np.empty([1,codewords.shape[0]])
        for i, word in enumerate(codewords):
            diff = np.linalg.norm(descriptor-word)**2
            descriptor_assignment[0][i] = np.exp(- a * diff)
        descriptor_assignment = descriptor_assignment/np.sum(descriptor_assignment)    
        histogram+=descriptor_assignment.reshape(descriptor_assignment.shape[1])
    histogram/=np.sum(histogram)

    return histogram


In [51]:
histogram = soft_assign_histogram(codewords, subregion_descriptors, a = 1)

In [56]:
for subregion in os.listdir('coordinates'):
    if 'sub_1' in subregion:
        subregion = np.load('coordinates/'+subregion)
        print(np.max(subregion.T[0]))

225


In [73]:
codewords = np.load('codewords.npy')


def create_global_Descriptor(scan_array):
    subregion_descriptor_list = []
    for subregion in os.listdir('coordinates'):
        if 'sub' in subregion:
            subregion_coordinates = np.load('coordinates/' + subregion)
            subregion_descriptors = []
            for coordinate in subregion_coordinates:
               subregion_descriptors.append(np.array(fn_hog_lbp_descriptor(coordinate, mri_scan)))

            subregion_descriptors = np.array(subregion_descriptors)
            histogram = soft_assign_histogram(codewords, subregion_descriptors, a = 0.4)   
            subregion_descriptor_list.append(histogram)
            
    global_descriptor = np.concatenate([subregion_descriptors[i] for i in range(len(subregion_descriptors))])
    global_descriptor/=3
    
    
    return global_descriptor

In [76]:
global_descriptor = create_global_Descriptor('data/9002430')

In [77]:
print(global_descriptor.shape, 
    np.max(global_descriptor), np.min(global_descriptor),
    global_descriptor)

(1932096,) 0.20529918564256247 0.0 [0.         0.08642463 0.         ... 0.03548001 0.03548001 0.01520572]


In [79]:
for i in range(200):
    print(global_descriptor[i])

0.0
0.0864246342354708
0.0
0.0
0.0
0.09687709951948675
0.0
0.0
0.0
0.08823992267409368
0.0
0.0
0.07195370267497235
0.0
0.0
0.0
0.05914940941565755
0.14150877274461918
0.09371924041480777
0.0
0.0
0.0
1.4947684432994876e-08
0.0
0.0527706876570846
0.0
0.0
0.0
0.13512961832117618
1.3458678695777655e-08
0.05387302145655564
0.13621673655006417
0.0805255214973465
0.0
0.0
0.0
0.0
0.0
0.0
0.0
7.050092764696394e-08
0.0
0.04670653993393591
0.021796385302503423
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.006227538657858121
0.021796385302503423
0.024910154631432483
0.06538915590751027
0.06850292523643932
0.05916161724965215
0.06538915590751027
0.0155688466446453
0.0155688466446453
0.024910154631432483
0.00934130798678718
0.00934130798678718
0.00934130798678718
0.006227538657858121
0.0031137693289290604
0.006227538657858121
0.006227538657858121
0.006227538657858121
0.012455077315716242
0.02175874116268589
0.005439685290671473
0.010879370581342945
0.010879370581342945
0.016319055872014415
0.010879370581342